In [ ]:
#https://github.com/wandb/examples/blob/master/examples/wandb-sweeps/sweeps-python/notebook.ipynb

In [1]:
# !pip install wandb -q

In [7]:
# !wandb login

'wandb'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [4]:
import wandb
import tensorflow as tf

In [6]:
wandb.init(project='wandb-tutorial',
           config={
               "layer_1_activation" : "relu",
               "layer_1" : 128,
               "learning_rate" : 0.01,
               "dropout_rate" : 0.2
           })

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ········


wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\MAEIL/.netrc


In [8]:
config = wandb.config

In [9]:
from wandb.keras import WandbCallback

In [10]:
mnist = tf.keras.datasets.mnist
(trian_x, train_y), (test_x, test_y) = mnist.load_data()

11493376/11490434 [==============================] - 1s 0us/step


In [11]:
trian_x, train_y =trian_x/255.0, train_y/255.0

In [12]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape= (28,28)),
    tf.keras.layers.Dense(config.layer_1, activation=config.layer_1_activation),
    tf.keras.layers.Dropout(config.dropout_rate),
    tf.keras.layers.Dense(10, activation='softmax')                                     
])

In [13]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = config.learning_rate),
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

In [16]:
model.fit(trian_x, train_y,  epochs= 5)

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0013 - accuracy: 0.0987
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0987
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0987
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0987
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0987


In [17]:
model.evaluate(test_x, test_y)

313/313 [==============================] - 1s 2ms/step - loss: 23184.7285 - accuracy: 0.0980


[23184.728515625, 0.09799999743700027]

### wandb.sweep

In [8]:
def train():
    import numpy as np
    import tensorflow as tf
    import wandb
    config_defaults = {
        'layers': 128,
        'dropout_rate' : 0.2,
        'layer1_activation' : 'relu',
        'optimizer' : 'adam',
        'learning_rate' : 0.01

    }
    wandb.init(project = 'sweep-pratice',
               config=config_defaults,
               magic=True)
    config = wandb.config

    fashion_mnist = tf.keras.datasets.fashion_mnist
    (train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
    class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
                   'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

    train_images.shape
    train_images = train_images / 255.0
    test_images = test_images / 255.0

    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(wandb.config.layers, activation=config.layer1_activation),
        tf.keras.layers.Dropout(config.dropout_rate),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])
    
    if config.optimizer =='rmsprop':
      opt = tf.keras.optimizers.RMSprop(learning_rate=config.learning_rate)
    else:
      opt = tf.keras.optimizers.Adam(learning_rate=config.learning_rate)

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    model.fit(train_images, train_labels, epochs=5,
                  validation_data=(test_images, test_labels))

In [9]:
sweep_config = {
    'method': 'grid',
    'parameters': {
        'layer1_size': {
            'values': [32, 64, 96, 128, 256]
        },
        'layer1_activation': {
            'values': ['relu', 'sigmoid']
        },
        'dropout_rate': {
            'values': [0.1, 0.2, 0.3, 0.4]
        },
        'optimizer': {
            'values': ['adam', 'rmsprop']
        },
        'learning_rate': {
            'values': [0.1, 0.01, 0.001]
        }
    }
}

In [10]:
import wandb
sweep_id = wandb.sweep(sweep_config, project= 'sweep-practics')

Create sweep with ID: 6nmpxc53
Sweep URL: https://wandb.ai/xyzmagic/sweep-practics/sweeps/6nmpxc53


In [18]:
wandb.agent(sweep_id, function=train)